In [1]:
#Import libraries

import gzip #for parsing gz files
import pandas as pd
import re 
import inflect #numeric/cardinal

In [17]:
dirPath = r"C:\Study\Projects\NLP\data"
#reviewPath = r"reviews_Tools_and_Home_Improvement_5.json.gz"
reviewPath = r"reviews_Musical_Instruments_5.json.gz"
qaPath = r"C:\Study\Projects\NLP\data" 
path = dirPath + "\\" + reviewPath

In [18]:
dataFile = gzip.open(path, 'r')

In [19]:
# Need this function because large amount of data we are going to load up

# http://jmcauley.ucsd.edu/data/amazon/index.html
# eval function allows text to be parsed as python code
# parse function is a generator with yield
# yield when we want to iterate over a sequence, but don't want to store the entire sequence in memory
def parse(dataFile):
    for each in dataFile:
        yield eval(each)

In [20]:
# for each review for a product in dataFile maps dict[nth] = review (nth)
# enumerate can't be used on a generator
count = 0
productReviews = {}
for eachReview in parse(dataFile):
    productReviews[count] = eachReview
    count = count + 1


In [21]:
len(productReviews)

10261

In [22]:
#productReviews[101718]

In [23]:
productsReviewData = pd.DataFrame.from_dict(productReviews, orient='index')

In [24]:
productsReviewData.shape
productsReviewData.columns
productsReviewData.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"


In [25]:
reduceData = productsReviewData.loc[:, ['asin', 'reviewerName', 'reviewText', 'helpful', 'overall']]




In [26]:
rating = lambda x: 1 if x > 2.0 else 0

In [27]:
reduceData['rating'] = reduceData['overall'].apply(rating)
unfavorableReviews = list(reduceData.loc[ reduceData['rating'] == 0, 
                                         ['reviewText', 'helpful', 'overall', 'rating']].sort_values(by='overall').index)
#unfavorableReviews
reduceData.loc[ reduceData['rating'] == 0, ['reviewText', 'helpful', 'overall', 'rating']].sort_values(by='overall').head(4)


,reviewText,helpful,overall,rating
5906,i got this despite the warnings about it being...,"[2, 2]",1.0,0
4405,"This thing is useless. Good idea, but way too ...","[0, 0]",1.0,0
4415,I've had it for about a month now and it hasn'...,"[0, 0]",1.0,0
4437,It's really not a 200w system. The booklet ev...,"[0, 0]",1.0,0


In [28]:
reviewTextProcessing = pd.DataFrame(reduceData.reviewText.copy())

In [29]:
reviewTextProcessing['rating'] = reduceData['rating']
reviewTextProcessing['overall'] = reduceData['overall']

In [30]:
productsReviewData.dtypes
len(productsReviewData)
# reviewText, rating, overall
reviewTextProcessing.shape

(10261, 3)

In [31]:
#reviewTextProcessing.loc[reviewTextProcessing['rating']==0, ['rating', 'overall', 'Processed', 'SentimentModel']]

In [33]:
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
import Contractions_Words as contractions
import importlib
importlib.reload(contractions)

contractions_dict = contractions.contractions_dict

#print(contractions_dict.keys())
#print(contractions_dict["s***"])

pattern = '%s' % '|'.join(contractions_dict.keys())
#pattern = r"%s" % pattern
#print(pattern)
contractions_re = re.compile(pattern)
def expand_contractions(s, contractions_dict=contractions_dict):
    #edit for special case like s*** etc
    s = re.sub("\*", "#", s)
    def replace(match):
        #print(type(match), match)
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)



In [34]:
sample1 = "s***"
print(re.sub("\*", "#", sample1))

sample = "I don\'t know what to say nay. I haven\'t. s###"
expand_contractions(sample)
expand_contractions(sample1)

s###


'shit'

In [35]:
from nltk.tokenize import word_tokenize, sent_tokenize
import string

# return both wholeText, paragraph Text
def singleReviewCleanUp(sample):
    #print(sample)
    #sample = sample.decode("utf-8")
    sample = expand_contractions(sample)
    sample = sample.lower()
    
    #for word in sample:
    #    print(lemmatizer.lemmatize(word, wordnet.VERB))
    
    review = sent_tokenize(sample)
    #print(review)
    for count, eachSent in enumerate(review):
        #remove numbers
        eachSent = re.sub("r'\d+/\d+|\d+|\$\d+.\d+", " ", eachSent)
        #remove string
        eachSent = re.sub('[%s]' % re.escape(string.punctuation), ' ', eachSent)
        #word tokens
        #print([lemmatizer.lemmatize(text, wordnet.VERB) for text in eachSent.split()])
        eachSent = word_tokenize(eachSent)
        review[count] = eachSent
        # lemmatize and remove stopWords
        #eachSent = [word for word in eachSent if not word in stopWords]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent if not word in stopWords]
        review[count] = " ".join(eachSent)
    wholeText = [" ".join(review) ]
    paragraphText = review
    #return wholeText
    #return paragraphText
    return paragraphText, wholeText

#paragraphText, wholeText = singleReviewCleanUp(sample)
#wholeText, paragraphText
#paragraphText

In [36]:
#Give me back text in a paragraph sentence forms. They do have more values than a clobbed of text based on Polarity Check.
cleanUpTextParagraph = lambda text: singleReviewCleanUp(text)[0]


In [37]:
%%time
reviewTextProcessing['Processed'] = reviewTextProcessing.reviewText.apply(cleanUpTextParagraph)


Wall time: 13.2 s


In [22]:
#sample = " I don't know what to do. I need to test through tough. Do you have any suggestions"
#result = cleanUpTextParagraph(sample)
#result

In [38]:
from textblob import TextBlob

# need to do a data check first
def getPolaritySubjectivity(data):
    #print(data)
    paragraphText = data
    sumPolarity = []
    sumSubjectivity = []
    for each in paragraphText:
        sumPolarity.append(TextBlob(each).sentiment.polarity)
        sumSubjectivity.append(TextBlob(each).sentiment.subjectivity)
    #print("sumPol", sumPolarity)
    return sumPolarity, sumSubjectivity
    #return sum(sumPolarity), sum(sumSubjectivity)

    #print(each, TextBlob(each).sentiment.polarity, TextBlob(each).sentiment.subjectivity)
#(sample2, TextBlob(sample2).sentiment.polarity, TextBlob(sample2).sentiment.subjectivity)


In [39]:
#Return the sume of polarity in a paragraph sentence when give a text in a list form of sentence

getPolarityFromText = lambda text: getPolaritySubjectivity(text)[0]

In [40]:
#for each in sampleText:
#    sample = cleanUpTextParagraph(each)
#    print(sample, getPolarityFromText(sample))

In [41]:
#print(stopWords)

In [42]:
#sentiment = lambda text: TextBlob(text).sentiment.polarity

In [43]:
adjustSentimentModel = lambda x: [sum(x), 0.0] if sum(x) < 0 else [sum(x), 1.0]

In [44]:
%%time
reviewTextProcessing['Sentiment']= reviewTextProcessing.Processed.apply(getPolarityFromText)

Wall time: 27.9 s


In [45]:
%%time
reviewTextProcessing['SentimentModel']= reviewTextProcessing.Sentiment.apply(adjustSentimentModel)

Wall time: 15 ms


In [46]:
#reviewTextProcessing.loc[unfavorableReviews, ['Processed', 'Sentiment', 'SentimentModel']]
#reviewTextProcessing.dtypes

In [47]:
reviewTextProcessing['CleanedJoin'] = reviewTextProcessing['Processed'].apply(', '.join)


In [32]:
reviewTextProcessing.shape
reviewTextProcessing.columns

Index(['reviewText', 'rating', 'overall', 'Processed', 'Sentiment',
       'SentimentModel', 'CleanedJoin'],
      dtype='object')

In [162]:
#from gensim import corpora
#dictionary = corpora.Dictionary(T)
#print(dictionary.token2id)

In [33]:
reviewTextProcessing['CleanedJoin'].head(5)

0    not much to write about here but it does exact...
1    the product does exactly as it should and is q...
2    the primary job of this device is to block the...
3    nice windscreen protects my mxl mic and preven...
4    this pop filter is great, it looks and perform...
Name: CleanedJoin, dtype: object

In [34]:

import itertools

largeText = reviewTextProcessing.CleanedJoin.to_list()
#largeText = reviewTextProcessing.reviewText.to_list() # Not too good accuracy around 116 missed with %86.94

len(largeText)

10261

In [37]:
sentiment = reviewTextProcessing['SentimentModel'].to_list()

In [36]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(largeText)
#tokenizer.word_index

In [241]:

size = int(len(largeText)*.75)
x_train_text = largeText[:size]
x_test_text = largeText[size:]

y_train =  reviewTextProcessing['rating'][:size]
y_test = reviewTextProcessing['rating'][size:]
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)


In [217]:
len(y_train), len(y_test), len(x_train_text), len(x_test_text)

(7695, 2566, 7695, 2566)

In [170]:
#x_train_tokens[1], x_train_text[1]

In [41]:
import numpy as np

x_train_text[1]
np.array(x_train_tokens[1])
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)


In [42]:
np.mean(num_tokens), np.max(num_tokens)


(92.37939771952051, 2047)

In [43]:
np.std(num_tokens)

114.14850119038401

In [44]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens) 
max_tokens


320

In [45]:
np.sum(num_tokens < max_tokens) / len(num_tokens)


0.9582886658220446

In [203]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

#from keras.models import MultinomialNB

In [175]:
import tensorflow as tf
#tf.__version__

In [279]:
from sklearn.feature_extraction.text import TfidfVectorizer
top_num_of_words = 5000
#vect = TfidfVectorizer(max_features=top_num_of_words, stop_words = 'english')
vect = TfidfVectorizer(max_features=top_num_of_words, stop_words = 'english', ngram_range=(1, 3), 
                       max_df=0.9, min_df=3, sublinear_tf=True)


In [280]:
text = x_train_text
vect.fit(text)
tfidf_train = vect.transform(text)


In [281]:
model = MultinomialNB()

In [209]:
#model = LinearRegression()

In [235]:
model = LogisticRegression(C=0.2, dual=True)


In [282]:
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [200]:
index = len(x_train_pad)
index

7695

In [50]:
%%time
model.fit(tfidf_train, y_train)
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=74)

NameError: name 'model' is not defined

In [284]:
%%time
vect.fit(x_test_text)
tfidf_test = vect.transform(x_test_text)


Wall time: 1.62 s


In [258]:
tfidf_test

<2566x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 91673 stored elements in Compressed Sparse Row format>

In [48]:
preds = model.predict(tfidf_test)
accuracy = (preds==y_test).mean()


NameError: name 'model' is not defined

In [49]:
print("Accuracy: {0:.2%}".format(accuracy))


NameError: name 'accuracy' is not defined

In [287]:
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
tn, fp, fn, tp

(16, 113, 210, 2227)

In [288]:
pd.DataFrame(y_pred).describe()

,0
count,1000.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [ ]:
#y_pred = y_pred.T[0]
#list(np.where(y_pred.T[0]))

In [ ]:
#y_pred

In [142]:
# No need with cls_pred as it has already been sorted.
cls_pred = np.array([1.0 if p > 0 else 0.1 for p in y_pred])
len(cls_pred)

1000

In [254]:
cls_true = np.array(y_test[0:1000])


In [253]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]
incorrect

array([  3,   5,  20,  23,  26,  36,  37,  40,  41,  42,  43,  54,  55,
        57,  58,  60,  61,  65,  67,  70,  72,  73,  75,  76,  79,  83,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  97, 100, 101, 106,
       107, 116, 124, 129, 130, 133, 134, 138, 143, 151, 153, 154, 155,
       156, 160, 165, 166, 168, 171, 175, 185, 190, 195, 196, 200, 201,
       205, 206, 207, 211, 214, 216, 217, 218, 219, 229, 232, 234, 235,
       237, 241, 243, 244, 247, 248, 249, 250, 257, 260, 266, 268, 271,
       272, 276, 280, 283, 287, 288, 290, 292, 294, 295, 296, 297, 299,
       301, 302, 306, 308, 309, 311, 312, 313, 314, 315, 317, 321, 323,
       326, 328, 333, 334, 335, 341, 342, 345, 348, 351, 352, 353, 354,
       359, 360, 361, 363, 367, 371, 372, 375, 376, 381, 382, 383, 387,
       388, 392, 394, 395, 396, 400, 401, 402, 407, 408, 411, 414, 415,
       416, 418, 426, 431, 432, 435, 441, 443, 444, 446, 447, 449, 451,
       452, 454, 459, 463, 464, 466, 468, 475, 477, 484, 488, 49

In [147]:
correct = np.where(cls_pred == cls_true)
correct = correct[0]
correct

array([  0,   1,   2,   4,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  21,  22,  24,  25,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  38,  39,  44,  45,  46,  47,  48,  49,
        50,  51,  52,  53,  56,  59,  62,  63,  64,  66,  68,  69,  71,
        74,  77,  78,  80,  81,  82,  84,  94,  95,  96,  98,  99, 102,
       103, 104, 105, 108, 109, 110, 111, 112, 113, 114, 115, 117, 118,
       119, 120, 121, 122, 123, 125, 126, 127, 128, 131, 132, 135, 136,
       137, 139, 140, 141, 142, 144, 145, 146, 147, 148, 149, 150, 152,
       157, 158, 159, 161, 162, 163, 164, 167, 169, 170, 172, 173, 174,
       176, 177, 178, 179, 180, 181, 182, 183, 184, 186, 187, 188, 189,
       191, 192, 193, 194, 197, 198, 199, 202, 203, 204, 208, 209, 210,
       212, 213, 215, 220, 221, 222, 223, 224, 225, 226, 227, 228, 230,
       231, 233, 236, 238, 239, 240, 242, 245, 246, 251, 252, 253, 254,
       255, 256, 258, 259, 261, 262, 263, 264, 265, 267, 269, 27

In [141]:
len(incorrect), len(correct)
(incorrect) (correct)

ValueError: operands could not be broadcast together with shapes (374,) (626,) 

In [148]:
result_location = np.where(cls_pred.T)[0] + index
len(result_location)

1000

In [149]:

NBresult = reviewTextProcessing.loc[list(result_location),  ['reviewText', 'overall', 'rating', 'Sentiment' , 'SentimentModel']]
#CNNresult.head(2)
#CNNresult.shape
#result_location
#result_location
NBresult.shape

(1000, 5)

In [150]:
NBresult['NB_Sentiment'] =  list(y_pred.T)


In [152]:
sumSentiment = lambda x: x[1]

In [153]:
NBresult.columns = ['TextBlobAnalysis' if x=='Sentiment' else x for x in NBresult.columns]
NBresult.columns = ['TextBlob_Sentiment' if x=='SentimentModel' else x for x in NBresult.columns]
NBresult['TextBlob_Sentiment'] = NBresult['TextBlob_Sentiment'].apply(sumSentiment)

In [154]:
NBresult.NB_Sentiment = NBresult.NB_Sentiment.astype(int)
NBresult.TextBlob_Sentiment = NBresult.TextBlob_Sentiment.astype(int)

In [155]:
NBresult.loc[NBresult.overall >= 3.0 , :].to_csv("Rating3.0above_NB.csv")
NBresult.loc[NBresult.overall < 3.0 , :].to_csv("RatingBelow3.0_NB.csv")

In [156]:
from sklearn.metrics import confusion_matrix

y_true = NBresult.rating.to_list()
NB_pred = NBresult.NB_Sentiment.to_list()
textBlob_pred = NBresult.TextBlob_Sentiment.to_list()


In [205]:
#textBlob_pred

In [158]:
tn, fp, fn, tp = confusion_matrix(y_true, NB_pred).ravel()
tn, fp, fn, tp

(20, 22, 352, 606)

In [159]:
tn, fp, fn, tp = confusion_matrix(y_true, textBlob_pred).ravel()
tn, fp, fn, tp

(14, 28, 57, 901)

# 

In [ ]:
pd.options.display.max_colwidth = 500

#productsReviewData.cleanedText.head(2)